<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Intro" data-toc-modified-id="Intro-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Intro</a></span></li><li><span><a href="#Implementation" data-toc-modified-id="Implementation-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Implementation</a></span></li><li><span><a href="#Examples" data-toc-modified-id="Examples-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Examples</a></span></li></ul></div>

# Abstract Factory

## Intro

An abstract factory provides an interface for creating families of related objects without specifying their concrete classes. Essentially, a factory of factories.

This pattern does not concern itself with the production of products, only that it can do so. 

This is done by using an abstract class that provides factory methods (Abstract Creator version of factory), providing encapsulation to a group of factories.

We should use this pattern when:
* when a system is composed of interdependent products
* has different families of objects that are used under different circumstances (different product interfaces)
* We need to handle multiple factory *types*
* When the system of products need to coexist


Some advantages to this pattern are:
* Isolates clients from concrete implementation
* Makes exchanging product families simpler
* promotes consistency amongst products

Some disadvantages of this pattern are:
* Rigid in terms of producing new types of products (Same as abstract creator)

## Implementation

In [92]:
interface Shape {
    void draw();
}

In [93]:
interface Colour {
    void fill();
}

In [94]:
class Red implements Colour {
    @Override
    public void fill() {
        System.out.println("This is Red");
    }
}

In [95]:
class Square implements Shape {
    @Override
    public void draw() {
        System.out.println("This is a square");
    }
}

In [96]:
abstract class AbstractFactory {
    abstract Colour getColour(String getColour);
    abstract Shape getShape(String getShape);
}

In [101]:
class ShapeFactory extends AbstractFactory {
    @Override
    public Colour getColour(String type){
        return null;
    }
    
    @Override
    public Shape getShape(String type){
        switch(type.toLowerCase()){
            case "square":
                return new Square();
            default:
                return null;
        }
    }
}

In [102]:
class ColourFactory extends AbstractFactory {
    @Override
    public Shape getShape(String type){
        return null;
    }
    
    @Override
    public Colour getColour(String type){
        switch(type.toLowerCase()){
            case "red":
                return new Red();
            default:
                return null;
        }
    }
}


So, this is basically the Abstract Creator version of the factory pattern but with one key difference. The Abstract class knows about the types of products it can create.

This example isn't following the interface segregation principle properly, so could lead to a fat abstract class. It would in fact make more sense to have a base implementation of both methods in the abstract class so the client could choose which to implement.

This would also allow the factory to satisfy Liskov substitution correctly, not that the factory will ever be passed into a function.

This of course is pretty messy and means factories will implement methods they do not need. Seems like using the Abstract Creator pattern is a better way forward here unless you want to implement a single factory that is responsible for creation of a family of objects.

To tie up this pattern, we need to implement a FactoryProducer

In [103]:
class FactoryProducer {
    public static AbstractFactory getFactory(String type)
    {
        switch(type.toLowerCase()){
            case "colour":
                return new ColourFactory();
            case "shape":
                return new ShapeFactory();
            default:
                return null;
        }
    }
}

In [104]:
AbstractFactory myShapeFactory = FactoryProducer.getFactory("shape");
Shape myShape = myShapeFactory.getShape("square");

In [108]:
myShape.draw();

This is a square


In [ ]:
AbstractFactory myColourFactory = FactoryProducer.getFactory("colour");
Colour myColour = myColourFactory.getColour("red");

In [107]:
myColour.fill();

This is Red


As shown, it is now possible for us to create factories of a given type. We use the static method here so that we don't need to concretise a class of the FactoryProducer (essentially follows the command pattern but without the naming).

## Examples

In [83]:
interface IMovie {
    void getMovieName();
}

In [84]:
abstract class AbstractMovieFactory {
    abstract IMovie getInstance(String type);
}

In [85]:
abstract class HollywoodMovie implements IMovie {
    public void getMovieName()
    {
        System.out.println("Hollywood Movie");
    }
}

In [86]:
abstract class BollywoodMovie implements IMovie {
    public void getMovieName()
    {
        System.out.println("Bollywood Movie");
    }
}

In [87]:
class HollywoodActionMovie extends HollywoodMovie {
    public void getMovieName()
    {
        System.out.println("Hollywood Action Movie");
    }
}

In [88]:
class HollywoodComedyMovie extends HollywoodMovie {
    public void getMovieName()
    {
        System.out.println("Hollywood Comedy Movie");
    }
}

In [89]:
class BollywoodActionMovie extends BollywoodMovie {
    public void getMovieName()
    {
        System.out.println("Bollywood Action Movie");
    }
}

In [90]:
class BollywoodComedyMovie extends BollywoodMovie {
    public void getMovieName()
    {
        System.out.println("Bollywood Comedy Movie");
    }
}

In [91]:
class HollywoodMovieFactory extends AbstractMovieFactory {
    public HollywoodMovie getInstance(String type)
    {
        switch(type.toLowerCase()){
            case "action":
                return new HollywoodActionMovie();
            case "comedy":
                return new HollywoodComedyMovie();
            default:
                return null;
        }
    }
}

In [92]:
class BollywoodMovieFactory extends AbstractMovieFactory {
    public BollywoodMovie getInstance(String type)
    {
        switch(type.toLowerCase()){
            case "action":
                return (BollywoodMovie) new BollywoodActionMovie();
            case "comedy":
                return (BollywoodMovie) new BollywoodComedyMovie();
            default:
                return null;
        }
    }
}

In [93]:
class FactoryProducer {
    public static AbstractMovieFactory getFactory(String type){
        switch(type.toLowerCase()){
            case "hollywood":
                return new HollywoodMovieFactory();
            case "bollywood":
                return new BollywoodMovieFactory();
            default:
                return null;                
        }
    }
}

In [95]:
AbstractMovieFactory hollywoodMovieFactory = 
                                        FactoryProducer.getFactory("hollywood");
                        
IMovie myMovie = hollywoodMovieFactory.getInstance("action");
myMovie.getMovieName();

Hollywood Action Movie


There is a lot going on here, but the key bit to remember is that the interface is what creates the 'contract' with the objects in question. The implementations of the abstract classes exist purely to *explain* how an object is meant to work once instantiated. The methods on this are also base implementations to ensure that subclasses can satisfy Liskov substitution if that was ever needed (Java enforces this anyway as you cannot use the abstract classes as types).

This pattern is a bit weird. Not sure when this would ever really be useful as concrete factories do basically the same thing. I guess this helps federate the code base and ensure people cannot misuse factories and objects.